In [1]:
from database import *

mdb = DatasetDB()
print(mdb.get_all_id())
print()
id = mdb.get_all_id()[0]
print(id)
print(mdb.get_config(id))

['ai_hub_korean_image_det', 'ai_hub_korean_image_rec', 'ICDAR2015', 'test1', 'test2']

ai_hub_korean_image_det
{'task': ['STD'], 'labelfiles': ['label.txt'], 'options': [], 'name': 'ai_hub_korean_image_det'}


In [12]:
from database import *
labeldb = LabelsetDB()
# labeldb.make("test", ["test1", "test2"], [16, 4, 80], random_seed=100)
labeldb.make("aihub_det_only_sign", ["ai_hub_korean_det"], [80, 10, 10], random_seed=100)
# name = labeldb.get_all_id()[0]
# print(name)
# print(labeldb.get_config("test"))

# labeldb.make_k_fold(["test"], "test_k_fold", 5)
# labeldb.make_k_fold(["test_k_fold_5_1", "test_k_fold_5_2"], "test_k_fold2", 5)
# labeldb.make_k_fold(["test_k_fold2_5_1", "test_k_fold2_5_2"], "test_k_fold2", 5)
# print(labeldb.get_config("test_k_fold2_5_1"))

# print(LabelsetDB().get("ai_hub_det_08_02_90_random_k_fold_5_1"))

train1\source\간판\세로형간판\간판_세로형간판_000001.jpg	[{"transcription": "\uc18c\ub9dd", "points": [[762, 306], [849, 306], [849, 518], [762, 518]]}, {"transcription": "\ubbf8\uc6a9\uc2e4", "points": [[754, 547], [837, 547], [837, 900], [754, 900]]}, {"transcription": "xxx", "points": [[730, 1053], [845, 1053], [845, 1147], [730, 1147]]}, {"transcription": "xxx", "points": [[871, 1078], [1047, 1078], [1047, 1139], [871, 1139]]}, {"transcription": "xxx", "points": [[1177, 732], [1479, 732], [1479, 794], [1177, 794]]}, {"transcription": "xxx", "points": [[1429, 707], [1502, 707], [1502, 748], [1429, 748]]}, {"transcription": "xxx", "points": [[1342, 1075], [1428, 1075], [1428, 1160], [1342, 1160]]}, {"transcription": "xxx", "points": [[184, 345], [361, 345], [361, 611], [184, 611]]}, {"transcription": "xxx", "points": [[121, 291], [184, 291], [184, 342], [121, 342]]}, {"transcription": "xxx", "points": [[365, 194], [460, 194], [460, 252], [365, 252]]}, {"transcription": "xxx", "points": [[391, 536]

In [3]:
from database import *
modeldb = ModelDB()
import os
# modeldb.get_config("en_PP-OCRv3_det")
modeldb.get_all_id()
modeldb.get_config("MobileNetV3_large_x0_5")
# for id in modeldb.get_all_id():
#     print(id)
#     modeldb.make_inference_model(id)

{'inference_model': './models/MobileNetV3_large_x0_5/inference_model/inference',
 'inference_model_dir': './models/MobileNetV3_large_x0_5/inference_model',
 'name': 'MobileNetV3_large_x0_5',
 'pretrained': 'pretrained_model'}

In [1]:
from database import *
from pathlib import Path
modeldb = ModelDB()


for name in modeldb.get_all_id():
    modeldb.check_config(name)

E:\workspace\paddleocr\models\en_PP-OCRv3_det\inference_model\Student\inference.pdmodel True
E:\workspace\paddleocr\models\en_PP-OCRv3_det\inference_model\Student True
E:\workspace\paddleocr\models\en_PP-OCRv3_det\inference_model\Student\inference.pdiparams True
E:\workspace\paddleocr\models\en_PP-OCRv3_det\pretrained_model True
E:\workspace\paddleocr\models\en_PP-OCRv3_det\pretrained_model\pretrained.pdparams True
en_PP-OCRv3_det False
E:\workspace\paddleocr\models\korean_PP-OCRv3_rec\inference_model\inference.pdmodel True
E:\workspace\paddleocr\models\korean_PP-OCRv3_rec\inference_model True
E:\workspace\paddleocr\models\korean_PP-OCRv3_rec\inference_model\inference.pdiparams True
E:\workspace\paddleocr\models\korean_PP-OCRv3_rec\pretrained_model True
E:\workspace\paddleocr\models\korean_PP-OCRv3_rec\pretrained_model\pretrained.pdparams True
korean_PP-OCRv3_rec False
E:\workspace\paddleocr\models\ml_PP-OCRv3_det\inference_model\Student\inference.pdmodel True
E:\workspace\paddleocr\mo

In [13]:
from database import *
workdb = WorkDB()
modeldb = ModelDB()
labelsetdb = LabelsetDB()

workdb.get_all_id()
workdb.make("test", ["aihub_det_only_sign"], "MobileNetV3_large_x0_5")
workdb.train("test", 10) 

In [4]:
ModelDB().get_config(ModelDB().get_all_id()[0])

{'name': 'en_PP-OCRv3_det', 'pretrained': 'pretrained'}

In [1]:
from database import *
import project
from pathlib import Path
config = "configs\det\det_mv3_db.yml"
labelsets = ["test", "test_k_fold_5_1"]
model = "MobileNetV3_large_x0_5"
name = "test"
WorkDB().make(name, labelsets, model, config)

WorkDB().train("test", 10)


'python E:\\workspace\\paddleocr\\code\\PaddleOCR\\tools\\train.py -c E:\\workspace\\paddleocr\\works\\test\\config.yml -o Global.epoch_num=10'

In [ ]:

from database import *
from pathlib import Path

def full_infer(image_dir, det_model_dir, rec_model_dir, save_dir):
    code = 'code/PaddleOCR/tools/infer/predict_system.py'
    det_algorithm="DB"

    # rec_model_dir = "E:/workspace/paddleocr/outputs/rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune/trained_model/inference_model"
    rec_char_dict_path = "E:/workspace/paddleocr/code/PaddleOCR/ppocr/utils/dict/korean_dict.txt"
    vis_font_path = "C:\Windows\Fonts\malgun.ttf"

    command = f"""python {code} --image_dir={image_dir} \
    --det_model_dir={det_model_dir} \
    --det_algorithm={det_algorithm} \
    --rec_model_dir={rec_model_dir} \
    --rec_image_shape="3, 32, 100" \
    --rec_char_dict_path={rec_char_dict_path} \
    --vis_font_path={vis_font_path} \
    --draw_img_save_dir={save_dir} \
    --drop_score=0.3 \
    --det_db_box_thresh=0.4"""
    
    path = Path(project.PROJECT_ROOT)/"inference/run.sh"
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "a") as f:
        f.write(command+"\n")
    
modeldb = ModelDB()
workdb = WorkDB()

# det_model_dir = modeldb.get_config("MobileNetV3_large_x0_5")["inference_model_dir"] 
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/Mobile_PaddleOCR_pretrained"
# full_infer(det_model_dir, rec_model_dir, save_dir)

# image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
# det_model_dir = modeldb.get_config("ml_PP-OCRv3_det")["inference_model_dir"]
# rec_model_dir = modeldb.get_config("korean_PP-OCRv3_rec")["inference_model_dir"]
# save_dir = f"{project.PROJECT_ROOT}/inference/PaddleOCR_PaddleOCR_pretrained_sangmu"
# full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

image_dir = "./origin_datasets/sangmu_real_image_det(cropped)"
det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_sangmu"
full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)

image_dir = "./datasets/ai_hub_korean_det_partial_test/images"
det_model_dir = workdb.get_config("det___ml_PP-OCRv3_det___ai_hub_det_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
rec_model_dir = workdb.get_config("rec___korean_PP-OCRv3_rec___rec_08_02_90_random_k_fold_5_1___default_config___finetune")["inference_model_dir"]
save_dir = f"{project.PROJECT_ROOT}/inference/PPOCR_PPOCR_tuned_aihub"
full_infer(image_dir, det_model_dir, rec_model_dir, save_dir)



In [4]:
from pathlib import Path

Path("./datasets/ai_hub_korean_image_det/train2/source/간판/돌출간판/돌출간판1/간판_돌출간판_001449.jpg").exists()

False

In [10]:
with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label2.txt") as f:
    lines = [line.rstrip("\n") for line in f.readlines()]

a, b, c = 0, 0, 0
for line in lines:
    if "가로형간판" in line:
        a+= 1
    elif "세로형간판" in line:
        b+= 1
    elif "돌출간판" in line:
        c+= 1
print(a, b, c)

80242 25621 55803


In [11]:
with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label3.txt", encoding="cp949") as f:
    with open("E:/workspace/paddleocr/datasets/ai_hub_korean_det/label2.txt", "w", encoding="utf-8") as f2:
        for line in f.readlines():
            f2.write(line)

In [4]:
from dataset.aihub.hangul_real_image_dataset import *
dataset = HangulRealImageDataset()


*******Checked Dir List*******
datasets\ai_hub_korean_det\train1\source\간판\세로형간판
datasets\ai_hub_korean_det\val1\source\간판\세로형간판
datasets\ai_hub_korean_det\train2\source\간판\가로형간판\가로형간판1
datasets\ai_hub_korean_det\train2\source\간판\돌출간판\돌출간판1
datasets\ai_hub_korean_det\train2\source\간판\세로형간판
datasets\ai_hub_korean_det\val2\source\간판\세로형간판
******************************


In [6]:
dataset = HangulRealImage_BoxDetectionDataset(dataset)

In [29]:
x, y = dataset[0]
dataset.show_xy(x, y)
y

[{'bbox': [[762, 306], [849, 306], [849, 518], [762, 518]], 'label': '소망'},
 {'bbox': [[754, 547], [837, 547], [837, 900], [754, 900]], 'label': '미용실'},
 {'bbox': [[730, 1053], [845, 1053], [845, 1147], [730, 1147]],
  'label': 'xxx'},
 {'bbox': [[871, 1078], [1047, 1078], [1047, 1139], [871, 1139]],
  'label': 'xxx'},
 {'bbox': [[1177, 732], [1479, 732], [1479, 794], [1177, 794]],
  'label': 'xxx'},
 {'bbox': [[1429, 707], [1502, 707], [1502, 748], [1429, 748]],
  'label': 'xxx'},
 {'bbox': [[1342, 1075], [1428, 1075], [1428, 1160], [1342, 1160]],
  'label': 'xxx'},
 {'bbox': [[184, 345], [361, 345], [361, 611], [184, 611]], 'label': 'xxx'},
 {'bbox': [[121, 291], [184, 291], [184, 342], [121, 342]], 'label': 'xxx'},
 {'bbox': [[365, 194], [460, 194], [460, 252], [365, 252]], 'label': 'xxx'},
 {'bbox': [[391, 536], [443, 536], [443, 587], [391, 587]], 'label': 'xxx'},
 {'bbox': [[355, 610], [517, 610], [517, 684], [355, 684]], 'label': 'xxx'},
 {'bbox': [[172, 748], [240, 748], [240, 